## 필요한 패키지 임포트 및 메소드 생성

In [182]:
## 패키지 설치
# pip install google-api-python-client

In [183]:
import pandas as pd
# anaconda환경 : conda install -c conda-forge google-api-python-client
from googleapiclient.discovery import build

def read_api_key():
    infile = open('C:/Users/4ser/Desktop/이도원/기타잡것/key_data/youtube_api_key_1.txt', 'r')
    api_key = infile.readline()
    return api_key

In [184]:
video_id = 'tZolEtoQyuY'
comments = list()
api_obj = build('youtube', 'v3', developerKey=read_api_key())

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/4ser/Desktop/이도원/기타잡것/key_data/youtube_api_key_1.txt'

## response

- response 1회당 1건씩 API 할당량 소모. while문은 maxResults 수치에 따라 다름. 예시) maxResults=100 : 할당량 약 4회
- part='snippet,replies'에서 snippet은 json파일의 최상단 key값이라 생각하면 됨. (필수조건)
- maxResults=10000은 최대로 가져올 데이터의 갯수를 10000개로 제한.
- 이렇게 가져온 response는 'items'라는 key값을 가지며, `response['items']`는 JSON파일로 나타냄.

In [ ]:
# https://www.youtube.com/watch?part='snippet,replies'&vieoid='asdasd'&maxResults=100이와 같이 요청을 함 snippet필수속성, replies댓글
response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=1000).execute() 

In [ ]:
while response:
    for item in response['items']:        
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

In [ ]:
response['items']

[{'kind': 'youtube#commentThread',
  'etag': 'cac_RZOGXNWPSAlOe_0JP0Be5Qk',
  'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
  'snippet': {'videoId': 'tZolEtoQyuY',
   'topLevelComment': {'kind': 'youtube#comment',
    'etag': '4xcAztyLhqHNQfGOQgvvH6KXa88',
    'id': 'Ugwm6rBc-G9J6j2tCRx4AaABAg',
    'snippet': {'videoId': 'tZolEtoQyuY',
     'textDisplay': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'textOriginal': '인트로 진짜 아기 시무룩했다가 다시 좋아하는거같네ㅋㅋㅋㅋㅋㅋ 왜이리커엽냐',
     'authorDisplayName': '트오밥',
     'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AL5GRJUC18-1DuTjvb8N-WmaM6LC-1iKONdQF4FwHw=s48-c-k-c0x00ffffff-no-rj',
     'authorChannelUrl': 'http://www.youtube.com/channel/UCrNbLJUDU5BE8tTU_JFL8gg',
     'authorChannelId': {'value': 'UCrNbLJUDU5BE8tTU_JFL8gg'},
     'canRate': True,
     'viewerRating': 'none',
     'likeCount': 9,
     'publishedAt': '2022-12-04T10:23:27Z',
     'updatedAt': '2022-12-04T10:23:27Z'}},
   'canReply': True,
   'totalReplyCount': 0,
   'isPublic': True}},
 {'ki

## 내보내기

### 데이터프레임화 및 csv로 내보내기

In [ ]:
df = pd.DataFrame(comments)
df.columns = ['comment', 'author', 'date', 'num_likes']
# df.to_csv('results.xlsx', index=None)
df

,comment,author,date,num_likes
0,"<a href=""https://www.youtube.com/watch?v=tZolE...",박민수,2023-02-12T13:12:25Z,0
1,"<a href=""https://www.youtube.com/watch?v=tZolE...",우름,2023-02-03T17:30:47Z,0
2,라꼰 시즌4 얼릉 해줘여 ㅠ,ㅎㅎㅎ,2023-02-03T02:19:01Z,0
3,먹방 방송중 자기들이 먹기 전에 스테프들 먼저 챙겨 주는 먹방은 처음보네요<br>침...,youngjin na,2023-02-02T21:27:53Z,1
4,아 진짜 만화가들의 저 상상력과 입담...쵝오야..끝까지 순수함을 지켜죠 ㅠ..,Punk-in Pie ASMR,2023-02-01T03:33:36Z,0
...,...,...,...,...
778,과연 몇이나 될까…,개삭수네,2022-12-14T09:05:53Z,0
779,장난치고 안먹으면 쳐맞아야 되지만<br>다먹음 ㅋㅋㅋㅋㅋㅋㅋ,Ne Beul,2022-12-07T08:33:24Z,1
780,하지만 맛있었죠?,두부,2022-12-05T14:57:31Z,2
781,탈모팸 1쉬,ㄷㅎㄱㅈㅇ,2022-12-05T05:47:38Z,0


In [ ]:
df.sort_values(by='num_likes', ascending=False)

,comment,author,date,num_likes
776,&quot;음식 가지고 장난치는 아재팸 1위&quot;,GeeChan Lee,2022-12-04T10:01:00Z,3323
568,다 커서도 이런 친구를 만날수 있다는게 넘 부럽다,크로커스,2022-12-04T10:51:57Z,1073
711,아 오늘 일요일이여서 기분 별로 였는데 저녁 보면서 보는 침착맨 개꿀잼이다...,물먹는미역,2022-12-04T10:05:22Z,1051
380,침펄이 풍한테 형형거리면서 노는거 왜케 귀엽냐,Cool태수,2022-12-04T13:41:27Z,1022
625,"<a href=""https://www.youtube.com/watch?v=tZolE...",강한별,2022-12-04T10:31:06Z,950
...,...,...,...,...
280,생신축하드려요,규민,2022-12-04T16:59:23Z,0
279,"<a href=""https://www.youtube.com/watch?v=tZolE...",XXX MADunknown,2022-12-04T17:02:44Z,0
278,"<a href=""https://www.youtube.com/watch?v=tZolE...",ㅡ,2022-12-04T17:09:13Z,0
277,칙촉이랑 제크 사오래요 왜케 얄밉냐ㅋㅋ,와플,2022-12-04T17:12:08Z,0


## JSON 파일로 내보내기

In [ ]:
import json

# response['items'] 변수
items = response['items']

# 저장할 파일 경로
file_path = './items.json'

# json 파일로 내보내기
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(items, json_file)

In [ ]:
import Preprocessing as prep

In [ ]:
import pandas as pd

# pd.DataFrame

prep.apply_regular_expression("그타6 언제 나오냐%^ 락스타 ㅊㅌ개새끼들!")

# prep.remove_one_letter_noun

AttributeError: module 'Preprocessing' has no attribute 'apply_regular_expression'

## 캐글 데이터셋

Mounted at /content/drive


In [20]:
## 파일 경로
## 문장
# file_path_train_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/train.hate.csv"
# file_path_dev_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/dev.hate.csv"
# file_path_unlabeled_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/unlabeled_comments.txt"
# file_path_no_hate = "/content/drive/MyDrive/공부/자연어 처리/자료/korean-hate-speech-detection/test.hate.no_label.csv"
# file_path_kold_v1 = "/content/drive/MyDrive/공부/자연어 처리/자료/kold_v1.json"
# file_path_youtube_comments = "/content/drive/MyDrive/공부/자연어 처리/자료/comments_update_df.csv"
# file_path_trip = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/tripadviser_review.csv"
# file_path_drama = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/Dataset.csv"
# file_path_common = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/common_word.csv"

# ## 단어
# file_path_pos_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/pos_word.csv"
# file_path_neg_word = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/neg_word.csv"
# file_path_unknown = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/obj_unknown_pol_word.txt"
# file_path_senti_dict = "/content/drive/MyDrive/공부/자연어 처리/자료/KnuSentiLex/SentiWord_Dict.txt"
file_path_stopwords = "C:/Users/TECH2_07/Desktop/이도원 프로젝트 폴더/자료/1차시도/stopwords.txt"

In [5]:
"""
[문장]
df_train_hate : 
 - label 있음.
 - none         3486
 - offensive    2499
 - hate         1911


df_dev_hate :
 - label 있음.
 - offensive    189
 - none         160
 - hate         122


df_unlabeled_comments : label 없음
df_no_hate : label 없음
df_kold_v1 : label 없음
df_youtube_comments : label 없음,  좋아요 수 있음


[단어]
pos_word.csv
neg_word.csv
obj_unknown_pol_word.txt
SentiWord_Dict.txt
"""

import pandas as pd


## 문장
# df_train_hate = pd.read_csv(file_path_train_hate, encoding='utf-8', engine='python')
# df_dev_hate = pd.read_csv(file_path_dev_hate, encoding='utf-8', engine='python')
# df_unlabeled_comments = pd.read_table(file_path_unlabeled_comments, encoding='utf-8', engine='python', names=['comment'],)
# df_no_hate = pd.read_csv(file_path_no_hate, encoding='utf-8', engine='python')
# df_kold_v1 = pd.read_json(file_path_kold_v1, encoding='utf-8')
# df_youtube_comments = pd.read_csv(file_path_youtube_comments, encoding='utf-8', engine='python')
df_trip = pd.read_csv(file_path_trip, encoding='utf-8', engine='python')
df_kdrama_common = pd.read_csv(file_path_common, encoding='utf-8', engine='python',)


## 단어
df_pos_word = pd.read_csv(file_path_pos_word, encoding='utf-8', engine='python')
df_neg_word = pd.read_csv(file_path_neg_word, encoding='utf-8', engine='python')
# df_unknown_word = pd.read_table(file_path_unknown, encoding='utf-8', engine='python', names=['word'])

In [6]:
def invert_rating(rating):
    y_value = 0
    if rating >= 4:
        y_value = 1
    elif rating == 3:
        y_value = 0
    elif rating <= 2:
        y_value = -1
    return y_value

In [7]:
import pandas as pd
import numpy as np

df_trip['y'] = df_trip.rating.apply(invert_rating)

df_pos_word.label = df_pos_word.word
df_pos_word.word = np.nan
df_pos_word.columns = ['rating', 'text', 'y']

df_neg_word.label = df_neg_word.word
df_neg_word.word = np.nan
df_neg_word.columns = ['rating', 'text', 'y']

new_df = pd.concat([df_trip, df_pos_word, df_neg_word], axis=0)
new_df = pd.concat([new_df, df_kdrama_common], axis=0)
new_df.drop(['rating'], axis=1, inplace=True)

new_df_m1 = new_df.loc[new_df.y == -1].sample(5300, random_state=33)
new_df_copy = new_df.loc[(new_df.y == 0) | (new_df.y == 1), :]
new_df = pd.concat([new_df_m1, new_df_copy], axis=0)
new_df.reset_index(drop=True, inplace=True)
new_df

,text,y
1315,몹시 슬프고,-1
9612,휘우듬하게 흔들리다,-1
3202,악한 짓을,-1
4387,하찮은 일로,-1
3555,옹졸한 모양,-1
...,...,...
5012,호랑이형 12년전 녹양동작업실앞에서 사진도찍어주시고 사인해주신거 정말감사했습니다.!,0
5013,어쩐대~~~김 정훈 진짜 진아 조아하던것가뜨만,0
5014,캬 큐브 먹여살린 현아를 퇴출이라니.. 초대형 FA나왔구나 모셔갈라고 경쟁 장난 아닐듯,0
5015,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?,0


In [8]:
new_df.y.value_counts()

 1    5572
-1    5300
 0    5172
Name: y, dtype: int64

In [9]:
new_df.isnull().sum()

text    0
y       0
dtype: int64

In [15]:
new_df

,text,y
1315,몹시 슬프고,-1
9612,휘우듬하게 흔들리다,-1
3202,악한 짓을,-1
4387,하찮은 일로,-1
3555,옹졸한 모양,-1
...,...,...
5012,호랑이형 12년전 녹양동작업실앞에서 사진도찍어주시고 사인해주신거 정말감사했습니다.!,0
5013,어쩐대~~~김 정훈 진짜 진아 조아하던것가뜨만,0
5014,캬 큐브 먹여살린 현아를 퇴출이라니.. 초대형 FA나왔구나 모셔갈라고 경쟁 장난 아닐듯,0
5015,이거 보다 더한 막장의 드라마도 넋놓고 보면서 무슨?,0


## 자연어 처리 시작

### 자연어 처리 모듈 임포트

In [68]:
import Preprocessing as prep
import Modeling as mo

['택시', '미만', '매우', '질문', '객실']

In [73]:
stopwords_list = ['제주도', '호스텔', '위치', '대중교통', '타고', '어디', '공항', '잠깐', '직원', '택시', '상품',]
stopwords_list = prep.stopwords(file_path_stopwords, stopwords_list)

In [74]:
prep.text_cleaning(new_df.text[5400], stopwords_list)

['미만', '매우', '질문', '객실', '매우', '손님', '투어', '할인', '모두', '다시', '다시']

### 문서-단어 행렬 생성

In [79]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer= lambda x: prep.text_cleaning(x, stopwords_list))
bow_vect = vect.fit_transform(new_df['text'].tolist())

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [84]:
word_list = vect.get_feature_names_out()
count_list = bow_vect.toarray().sum(axis=0)
count_list

array([ 5,  1, 15, ...,  1,  1,  1], dtype=int64)

In [85]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict

{'가가': 5,
 '가가린': 1,
 '가게': 15,
 '가격': 247,
 '가격표': 1,
 '가관': 2,
 '가구': 12,
 '가급': 1,
 '가기': 22,
 '가까이': 23,
 '가끔': 9,
 '가나': 4,
 '가난': 5,
 '가난뱅이': 1,
 '가념하': 1,
 '가능': 12,
 '가능성': 4,
 '가다가': 1,
 '가당': 1,
 '가도': 7,
 '가동': 2,
 '가두': 1,
 '가드': 1,
 '가드올려': 1,
 '가득': 5,
 '가든': 2,
 '가라': 5,
 '가락': 2,
 '가래': 2,
 '가량': 5,
 '가렴': 1,
 '가렵': 1,
 '가로': 7,
 '가로본능폰': 1,
 '가로세로': 3,
 '가로수길': 1,
 '가루': 2,
 '가르침': 3,
 '가마니': 1,
 '가면': 21,
 '가무': 1,
 '가문': 1,
 '가물': 1,
 '가뭄': 2,
 '가미': 1,
 '가방': 4,
 '가버': 1,
 '가보': 1,
 '가봉': 1,
 '가빠': 1,
 '가쁘': 3,
 '가쁜': 1,
 '가사': 5,
 '가성': 50,
 '가세': 9,
 '가소': 1,
 '가수': 29,
 '가스': 1,
 '가스레인지': 1,
 '가스렌지': 1,
 '가슴': 19,
 '가슴앓이': 1,
 '가시': 4,
 '가식': 2,
 '가신': 3,
 '가안': 1,
 '가야': 20,
 '가엽다': 2,
 '가오': 1,
 '가옥': 1,
 '가요': 16,
 '가요무대': 1,
 '가요제': 1,
 '가용': 2,
 '가운데': 4,
 '가운데손가락': 1,
 '가위': 1,
 '가을': 5,
 '가인': 3,
 '가장': 54,
 '가점': 1,
 '가정': 14,
 '가정불화': 1,
 '가정사': 2,
 '가정식': 2,
 '가정폭력': 3,
 '가족': 132,
 '가족사': 1,
 '가족사진': 1,
 '가족영화': 2,
 '가즈': 11,
 '가증': 1,
 '가지': 73,
 '가짐'

### TF-IDF 생성

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
tfidf_vect = tfidf.fit_transform(bow_vect)
tfidf_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### 단어 맵핑

In [90]:
invert_index_vectorizer = {v : k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer[2928]

'몹시'

In [88]:
vect.vocabulary_

{'몹시': 2928,
 '악한': 5439,
 '일로': 7060,
 '모양': 2868,
 '매우': 2693,
 '위태롭거': 6513,
 '부정': 3809,
 '구차': 885,
 '흥글': 10149,
 '방망이': 3386,
 '생기': 4348,
 '무효': 3025,
 '냄새': 1543,
 '침구': 8705,
 '침대': 8706,
 '시트': 5111,
 '곰팡이': 641,
 '하나': 9475,
 '포기': 9299,
 '외국인': 6253,
 '국가': 895,
 '망신': 2659,
 '생각': 4344,
 '주말': 7924,
 '방이': 3404,
 '다른': 1740,
 '숙박': 4850,
 '업체': 5788,
 '느낌': 1697,
 '가세': 54,
 '움찔': 6407,
 '토라': 9015,
 '고충': 602,
 '괴로움': 797,
 '쇠멸': 4745,
 '강제': 221,
 '궁리': 962,
 '자꾸': 7193,
 '짜증': 8253,
 '침체': 8712,
 '분란': 3847,
 '체면': 8475,
 '기소': 1147,
 '지경': 8062,
 '조금': 7770,
 '연줄': 5975,
 '현상': 9815,
 '사람': 4124,
 '반함': 3321,
 '마음': 2547,
 '배은망덕': 3443,
 '수치': 4833,
 '심술': 5263,
 '종기': 7871,
 '욕심': 6310,
 '고소': 568,
 '공포증': 715,
 '멸시': 2799,
 '정처': 7695,
 '자릿': 7204,
 '숨쉬기': 4883,
 '명망': 2812,
 '소름': 4642,
 '이야기': 6871,
 '행동': 9732,
 '오염': 6132,
 '식품': 5138,
 '함부로': 9638,
 '마구': 2502,
 '의사': 6721,
 '방정': 3408,
 '비만': 3987,
 '개비': 265,
 '주책': 7966,
 '능청': 1716,
 '얼굴빛': 5750,
 '누르렀다': 1

## 모델링

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

In [98]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_vect, new_df['y'], shuffle=True, test_size=0.2, stratify=new_df.y)

model = RandomForestClassifier(random_state=33)
y_pred = model.fit(x_train, y_train).predict(x_test)


In [100]:
accuracy_score(y_pred, y_test)

0.6974135244624493

In [ ]:
None, 'micro', 'macro', 'weighted', 'samples'

In [113]:
recall_score(y_pred, y_test, average='macro')

0.7526329297602062

In [114]:
precision_score(y_pred, y_test, average='macro')

0.7025907944507278

In [115]:
f1_score(y_pred, y_test, average='macro')

0.694539721052562

In [104]:
confusion_matrix(y_pred, y_test)

array([[963, 178, 590],
       [ 22, 794,  44],
       [ 75,  62, 481]], dtype=int64)